In [46]:
import numpy as np
import pandas as pd
import re
import time
import pickle
import pprint
from glob import glob

from selenium import webdriver

from hackaroutine import Ingredient, Product, Routine

In [2]:
# CHROMEDRIVER_PATH = '/Users/Jelly/Documents/googledrive/chromedriver/macos/chromedriver'
CHROMEDRIVER_PATH = 'C:/chromedriver_win32/chromedriver.exe'

In [3]:
driver = webdriver.Chrome(CHROMEDRIVER_PATH)

In [4]:
file_list = glob('./data/*.csv')
file_list

['./data\\brands.csv',
 './data\\recommendations_2020-05-05.csv',
 './data\\responses_2020-05-05.csv',
 './data\\responses_modified_2020-05-05.csv']

In [5]:
responses = pd.read_csv(file_list[3])
responses.head(2)

,Timestamp,What's your skin type?,Do you have any known allergies? Or any ingredients you avoid for any reason?,What are your top skin concerns?,What is your current day routine? (Please list your products in order from first to last),What is your current night routine (Please list your products in order from first to last),What type of summer weather will you have?,Any known skin sensitivities?,"Any experience with retinoids? (If so, please tell us your use history)","Any experience with acids? (If so, please tell us your use history, what type, and any known concentrations)","Are you prone to breakouts? If so, how severe are the breakouts and what is your acne treatment history?",Anything else you want the chemists to know about your skin and your skincare routine?,"Can we share your routine learnings (anonymously) on our social media (post, podcast, or insta live shoutout)?"
0,4/30/2020 12:43:32,Combination,None,"Acne, Pigmentation, Fine lines/wrinkles","trader joe's all in one cleanser, bioderma seb...","trader joe's all in one cleanser, Bioderma Seb...","Temperate, a lovely 70F (lucky!)",I broke out from Cosrx snail serum and from Ma...,Differing 0.1% for four years,I prefer toners with acids as masks with AHA s...,I have mild to moderate acne that I have been ...,"I have a super oily forehead, oily nose and c...",Yes
1,4/30/2020 12:45:25,Combination,No allergies,"Acne, Fine lines/wrinkles, Skin Texture (rough...","Biore Charcoal Acne Clearing Cleanser, Kiel's ...",NaN,"Tropical, humid",No.,No.,No.,Yes. They are usually cystic and around my cyc...,I need help.,Yes


In [6]:
new_columns = [
    'timestamp',
    'skin_type',
    'cannot_contain',
    'skin_concern',
    'am_routine',
    'pm_routine',
    'climate',
    'skin_sensitivities',
    'used_retinoids',
    'used_acids',
    'prone_to_breakouts',
    'miscellaneous',
    'permission'
]

responses.columns = new_columns
responses.head(2)

,timestamp,skin_type,cannot_contain,skin_concern,am_routine,pm_routine,climate,skin_sensitivities,used_retinoids,used_acids,prone_to_breakouts,miscellaneous,permission
0,4/30/2020 12:43:32,Combination,None,"Acne, Pigmentation, Fine lines/wrinkles","trader joe's all in one cleanser, bioderma seb...","trader joe's all in one cleanser, Bioderma Seb...","Temperate, a lovely 70F (lucky!)",I broke out from Cosrx snail serum and from Ma...,Differing 0.1% for four years,I prefer toners with acids as masks with AHA s...,I have mild to moderate acne that I have been ...,"I have a super oily forehead, oily nose and c...",Yes
1,4/30/2020 12:45:25,Combination,No allergies,"Acne, Fine lines/wrinkles, Skin Texture (rough...","Biore Charcoal Acne Clearing Cleanser, Kiel's ...",NaN,"Tropical, humid",No.,No.,No.,Yes. They are usually cystic and around my cyc...,I need help.,Yes


In [7]:
routine_columns = ['am_routine', 'pm_routine']

for col in routine_columns:
    responses[col] = responses[col].str.lower().str.replace(';', ',').str.split(r'\,+\s*')
    print(responses[col][:2])

0    [trader joe's all in one cleanser, bioderma se...
1    [biore charcoal acne clearing cleanser, kiel's...
Name: am_routine, dtype: object
0    [trader joe's all in one cleanser, bioderma se...
1                                                  NaN
Name: pm_routine, dtype: object


In [8]:
responses['am_routine'][0]

["trader joe's all in one cleanser",
 'bioderma sebium h2o',
 'the ordinary ascorbic acid 8%',
 'bioderma sebium mat control cream',
 'purlisse bb cream']

In [9]:
# prior experience

popular_brands = [
    'La Roche Posay',
    'Cerave', 
    'Cetaphil', 
    'Timeless Skincare', 
    'Skinceuticals', 
    'Stratia Skincare', 
    'Paula’s Choice', 
    'CosRX', 
    'The Ordinary', 
    'Drunk Elephant', 
    'Klairs', 
    'Inkey List', 
    'Farmacy ', 
    'First Aid Beauty'
]

popular_ingredients = [
    'Vitamin C', 
    'Glycolic Acid', 
    'Lactic Acid', 
    'Mandelic Acid', 
    'Niacinamide', 
    'Retinol', 
    'Hydroxypinocolone Retinoate', 
    'Bakuchiol', 
    'Retinyl Palmitate', 
    'Ceramides', 
    'Salicylic Acid', 
    'Willow bark Extract', 
    'Adapalene', 
    'Sodium Lauryl Sulfate', 
    'Cocoamidopropyl Betaine', 
]

In [48]:
tick = time.time()
n = 2
routine = Routine(name=n, routine=responses['am_routine'][n])
routine.link_sync()
print(routine.routine)
tock = time.time()
print(f'{tock - tick} s')

['drunk elephant beste no. 9 jelly cleanser', "paula's choice skin balancing pore-reducing toner", "paula's choice boost 10% niacinamide booster", "paula's choice boost 10% azelaic acid booster", 'cerave pm facial moisturizing lotion', 'eltamd uv clear broad-spectrum spf 46']
7.4957115650177 s


In [50]:
routine.top

[('glycerin', 5),
 ('phenoxyethanol', 5),
 ('water', 4),
 ('butylene glycol', 4),
 ('niacinamide', 4),
 ('xanthan gum', 4),
 ('sodium hyaluronate', 3),
 ('propanediol', 2),
 ('dipotassium glycyrrhizate', 2),
 ('citric acid', 2)]

In [53]:
routine.has('1,3-butanediol')

Routine does not have 1,3-butanediol.


[]

In [12]:
routine.routine

['drunk elephant beste no. 9 jelly cleanser',
 "paula's choice skin balancing pore-reducing toner",
 "paula's choice boost 10% niacinamide booster",
 "paula's choice boost 10% azelaic acid booster",
 'cerave pm facial moisturizing lotion',
 'eltamd uv clear broad-spectrum spf 46']

In [13]:
for ingredient in popular_ingredients:
    print(ingredient)
    print(routine.has(ingredient))
    print()

Vitamin C
Routine does not have Vitamin C.
[]

Glycolic Acid
Routine does not have Glycolic Acid.
[]

Lactic Acid
['eltamd uv clear broad-spectrum spf 46']

Mandelic Acid
Routine does not have Mandelic Acid.
[]

Niacinamide
["paula's choice skin balancing pore-reducing toner", "paula's choice boost 10% niacinamide booster", 'cerave pm facial moisturizing lotion', 'eltamd uv clear broad-spectrum spf 46']

Retinol
Routine does not have Retinol.
[]

Hydroxypinocolone Retinoate
Routine does not have Hydroxypinocolone Retinoate.
[]

Bakuchiol
Routine does not have Bakuchiol.
[]

Retinyl Palmitate
Routine does not have Retinyl Palmitate.
[]

Ceramides
Routine does not have Ceramides.
[]

Salicylic Acid
["paula's choice boost 10% azelaic acid booster"]

Willow bark Extract
Routine does not have Willow bark Extract.
[]

Adapalene
Routine does not have Adapalene.
[]

Sodium Lauryl Sulfate
Routine does not have Sodium Lauryl Sulfate.
[]

Cocoamidopropyl Betaine
['drunk elephant beste no. 9 jelly

## string grouper demo

string_grouper is giving me a hard time, so i am going to do this manually according to:
https://bergvca.github.io/2017/10/14/super-fast-string-matching.html

In [14]:
import json
from string_grouper import match_strings, match_most_similar, group_similar_strings, StringGrouper

In [15]:
with open('./data/big_dict.json', 'rb') as handle:
    big_dict = json.load(handle)

In [16]:
product_list = pd.read_json('./data/big_dict.json', orient='index').reset_index(level=0)
product_list.columns = ['name', 'brand', 'product']
product_list.head()

,name,brand,product
0,kiehl's creme de corps,kiehl's,creme de corps
1,kiehl's bath and shower liquid body cleanser,kiehl's,bath and shower liquid body cleanser
2,kiehl's body fuel,kiehl's,body fuel
3,kiehl's ultimate strength hand salve,kiehl's,ultimate strength hand salve
4,kiehl's ultra facial cream,kiehl's,ultra facial cream


In [35]:
product_list['name']

0                                 kiehl's creme de corps
1           kiehl's bath and shower liquid body cleanser
2                                      kiehl's body fuel
3                   kiehl's ultimate strength hand salve
4                             kiehl's ultra facial cream
                             ...                        
168                      la roche-posay my skin track uv
169    la roche-posay toleriane ultra makeup remover ...
170           la roche-posay micellar water gel cleanser
171    la roche-posay respectissime extension lengthe...
172         la roche-posay respectissime eyeliner pencil
Name: name, Length: 173, dtype: object

In [24]:
# matches = match_most_similar(product_list['name'], sample_list)
# pd.DataFrame({'sample_list': sample_list, 'matches': matches})

In [32]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [28]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\s',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
matrix = vectorizer.fit_transform(sample_list)

In [31]:
print(matrix[0])
print(ngrams(sample_list[0]))

(0, 111)	0.1331105824299741
  (0, 58)	0.1331105824299741
  (0, 20)	0.1331105824299741
  (0, 11)	0.1331105824299741
  (0, 89)	0.1331105824299741
  (0, 71)	0.1331105824299741
  (0, 88)	0.10494591211268606
  (0, 30)	0.1331105824299741
  (0, 118)	0.1331105824299741
  (0, 75)	0.1331105824299741
  (0, 23)	0.1331105824299741
  (0, 0)	0.1331105824299741
  (0, 7)	0.1331105824299741
  (0, 6)	0.1331105824299741
  (0, 5)	0.1331105824299741
  (0, 44)	0.1331105824299741
  (0, 119)	0.1331105824299741
  (0, 33)	0.1331105824299741
  (0, 106)	0.10494591211268606
  (0, 124)	0.10494591211268606
  (0, 100)	0.1331105824299741
  (0, 55)	0.10494591211268606
  (0, 37)	0.1331105824299741
  (0, 90)	0.1331105824299741
  (0, 102)	0.1331105824299741
  :	:
  (0, 93)	0.1331105824299741
  (0, 72)	0.1331105824299741
  (0, 17)	0.1331105824299741
  (0, 65)	0.1331105824299741
  (0, 120)	0.1331105824299741
  (0, 64)	0.1331105824299741
  (0, 98)	0.1331105824299741
  (0, 54)	0.1331105824299741
  (0, 109)	0.1331105824299741
 

In [39]:
# https://caserta.com/data-blog/string-matching-record-linkage-python-strategies/

def tfidf_match(list1, list2):
    """For each item in list1, find the match in list2"""
    vectorizer = TfidfVectorizer(analyzer=ngrams, lowercase=True)
    tfidf = vectorizer.fit_transform(list2)
    nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
    distances, indices = nbrs.kneighbors(vectorizer.transform(list1))
    
    matches = [(round(distances[i][0], 2), list1[i], list2[j[0]]) 
               for i, j in enumerate(indices)]
    matches = pd.DataFrame(matches, 
                           columns=['score', 'input', 'matched'])
    return matches

In [52]:
tick = time.time()
kiehls = [
    "Kiel's Powerful-Strength Line-Reducing Concentrate 12.5% Vitamin C Serum",
    "Kiehl's Calendula hydrating mask 5 minute",
    "Kiehl's Blue Herbal Acne Cleansing Treatment",
    "Kiehl's ultrafacial toner"
]
matches = tfidf_match(kiehls, product_list['name'])
tock = time.time()
print(f'{tock - tick} s')
matches

0.11469340324401855 s


,score,input,matched
0,1.00,Kiel's Powerful-Strength Line-Reducing Concent...,kiehl's powerful-strength vitamin c serum
1,1.03,Kiehl's Calendula hydrating mask 5 minute,kiehl's calendula & aloe soothing hydration mask
2,0.95,Kiehl's Blue Herbal Acne Cleansing Treatment,kiehl's blue herbal acne cleanser treatment
3,0.12,Kiehl's ultrafacial toner,kiehl's ultra facial toner


In [43]:
lrp = [
    "La Roche-Posay tinted Anthelios 50 Mineral Sunscreen",
    "La Roche Posay Medicated Gel Cleanser",
    "La Roche-Posay Tolerane Hydrating Cleanser"
]
tfidf_match(lrp, product_list['name'])

,score,input,matched
0,0.88,La Roche-Posay tinted Anthelios 50 Mineral Sun...,la roche-posay anthelios tinted mineral sunscr...
1,1.05,La Roche Posay Medicated Gel Cleanser,la roche-posay effaclar medicated acne face wash
2,1.02,La Roche-Posay Tolerane Hydrating Cleanser,la roche-posay toleriane hydrating gentle faci...


In [56]:
english = [
    'first i use la roche posay effaclare cleanser', 
    'then moisturize using kiehls super multi corrective cream',
    'then kiehls facial fuel sunblock if i go outside']

tfidf_match(english, product_list['name'])

,score,input,matched
0,1.07,first i use la roche posay effaclare cleanser,la roche-posay effaclar cream cleanser for oil...
1,0.67,then moisturize using kiehls super multi corre...,kiehl's super multi-corrective cream
2,0.94,then kiehls facial fuel sunblock if i go outside,kiehl's facial fuel spf 20


## common ingredient dictionary

In [ ]:
with open('./data/master_dict.pickle', 'rb') as handle:
    master_dict = pickle.load(handle)

In [ ]:
def ingredient_dict_builder(routine_dict, master_dict):
    base_url = 'https://cosdna.com/eng/'
    filtered_dict = {k:v for k,v in routine_dict.items() if k not in master_dict.keys()}
    for k, v in filtered_dict.items():
        driver.get(base_url + k + '.html')
        print(f'{k}: {v}')
        ingredient = input('preferred name: ')
        if ingredient == 'BREAK':
            break
        elif ingredient == '':
            master_dict[k] = v
        else:
            master_dict[k] = ingredient
    return master_dict

In [ ]:
am_routines = []
for rout in responses['am_routine']:
    routine = Routine(name=i, routine=responses['am_routine'][i])
    routine.link_sync()
#     master_dict = ingredient_dict_builder(routine._routine_dict, master_dict)
    am_routines.append(routine)

In [ ]:
for i in range(5, 10):
    routine = Routine(name=i, routine=responses['am_routine'][i])
    routine.link_sync()
    master_dict= ingredient_dict_builder(routine._routine_dict, master_dict)

In [ ]:
master_dict

In [ ]:
# saving pickle
with open('./data/master_dict.pickle', 'wb') as handle:
    pickle.dump(master_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)